In [ ]:
drive.flush_and_unmount()


In [1]:
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


14.7 - save h images with the mask

In [ ]:
import cv2
import numpy as np
import os

def detect_circular_shape(image_path, output_dir, scale_factor=0.2, min_radius=300, max_radius=400):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Check if the image was loaded successfully
    if image is None:
        print(f"Error loading image: {image_path}")
        return False

    # Resize the image
    resized_image = cv2.resize(image, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)

    # Enhance contrast using histogram equalization
    enhanced_image = cv2.equalizeHist(resized_image)

    # Apply Gaussian Blur to reduce noise
    blurred_image = cv2.GaussianBlur(enhanced_image, (9, 9), 2)

    # Apply the Hough Circle Transform on the resized image
    circles = cv2.HoughCircles(
        blurred_image,
        cv2.HOUGH_GRADIENT,
        dp=1,
        minDist=100,  # Adjust based on the distance between expected circles
        param1=100,
        param2=50,  # Adjusted to be less strict
        minRadius=int(min_radius * scale_factor),
        maxRadius=int(max_radius * scale_factor)
    )

    # Ensure at least some circles are found
    if circles is not None:
        circles = np.uint16(np.around(circles))
        valid_circles = []

        for circle in circles[0, :]:
            center_x = int(circle[0] / scale_factor)
            center_y = int(circle[1] / scale_factor)
            radius = int(circle[2] / scale_factor)

            if is_circle_valid(image, center_x, center_y, radius):
                valid_circles.append((center_x, center_y, radius))

        if valid_circles:
            # Select the largest valid circle
            largest_circle = max(valid_circles, key=lambda x: x[2])

            center_x, center_y, radius = largest_circle

            # Reload the original color image
            original_image = cv2.imread(image_path)

            # Create a mask with the circle
            mask = np.zeros_like(original_image)
            cv2.circle(mask, (center_x, center_y), radius, (255, 255, 255), thickness=-1)

            # Apply the mask to the original image
            result_image = cv2.bitwise_and(original_image, mask)

            # Convert grayscale image to BGR for colored drawing
            image_color = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

            # Draw the outer circle in red
            cv2.circle(image_color, (center_x, center_y), radius, (0, 0, 255), 4)  # Red color, thicker line
            # Draw the center of the circle
            cv2.circle(image_color, (center_x, center_y), 2, (0, 255, 0), 3)

            # Save the result image with the circle
            output_image_path = os.path.join(output_dir, os.path.basename(image_path))
            cv2.imwrite(output_image_path, image_color)

            # Save the result image with only the circle
            result_output_path = os.path.join(output_dir, f"circle_{os.path.basename(image_path)}")
            cv2.imwrite(result_output_path, result_image)

            return True
        else:
            print(f"No valid circles were found in {os.path.basename(image_path)}")
    else:
        print(f"No circles were found in {os.path.basename(image_path)}")

    return False

def is_circle_valid(image, center_x, center_y, radius):
    # Add any additional criteria for validating circles here
    # For example, check if the circle is within the image bounds
    h, w = image.shape
    return True

# Path to the images directory
images_directory = '/content/mydrive/Othercomputers/My PC/Thesis/roboflowV6_1169/original/bounded'
output_directory = '/content/mydrive/Othercomputers/My PC/Thesis/roboflowV6_1169/circeled_images_25.7'
os.makedirs(output_directory, exist_ok=True)

# Process all images starting with "LBP" in the directory
image_files = [f for f in os.listdir(images_directory) if f.startswith("LBP") and f.endswith(".JPG")]
total_images = len(image_files)
total_circles = 0

for image_file in image_files:
    image_path = os.path.join(images_directory, image_file)
    detected = detect_circular_shape(image_path, output_directory, scale_factor=0.2, min_radius=1200, max_radius=1700)
    if detected:
        total_circles += 1

# Final summary
print(f"\nTotal number of images: {total_images}")
print(f"Total circles detected: {total_circles}")



Total number of images: 0
Total circles detected: 0


15.7 -find the red circle on the lbp mandarin, then take the original image, compare the name to the lbp_images, and mask the images and keep only the mandarin from the original images

In [2]:
import cv2
import numpy as np
import os

def find_original_image(image_name):
    original_image_name = image_name.replace("LBP_Thresh_", "")
    search_paths = [
        '/content/mydrive/Othercomputers/My PC/Thesis/roboflowV6_1169/original/bounded'
    ]

    for path in search_paths:
        original_image_path = os.path.join(path, original_image_name)
        print(f"Searching for {original_image_path}")
        if os.path.exists(original_image_path):
            return original_image_path
    return None

def detect_circular_shape(image_path, output_dir, scale_factor=0.2, min_radius=300, max_radius=400):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Error loading image: {image_path}")
        return False

    # Resize the image
    resized_image = cv2.resize(image, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)

    # Enhance contrast using histogram equalization
    enhanced_image = cv2.equalizeHist(resized_image)

    # Apply Gaussian Blur to reduce noise
    blurred_image = cv2.GaussianBlur(enhanced_image, (9, 9), 2)

    # Apply the Hough Circle Transform on the resized image
    circles = cv2.HoughCircles(
        blurred_image,
        cv2.HOUGH_GRADIENT,
        dp=1,
        minDist=100,  # Adjust based on the distance between expected circles
        param1=100,
        param2=50,  # Adjusted to be less strict
        minRadius=int(min_radius * scale_factor),
        maxRadius=int(max_radius * scale_factor)
    )

    # Ensure at least some circles are found
    if circles is not None:
        circles = np.uint16(np.around(circles))
        valid_circles = []

        for circle in circles[0, :]:
            center_x = int(circle[0] / scale_factor)
            center_y = int(circle[1] / scale_factor)
            radius = int(circle[2] / scale_factor)

            if is_circle_valid(image, center_x, center_y, radius):
                valid_circles.append((center_x, center_y, radius))

        if valid_circles:
            # Select the largest valid circle
            largest_circle = max(valid_circles, key=lambda x: x[2])

            center_x, center_y, radius = largest_circle

            # Find and load the original color image
            original_image_name = os.path.basename(image_path).replace("LBP_Thresh_", "")
            original_image_path = find_original_image(original_image_name)
            print("original_image_path", original_image_path)
            if original_image_path is None:
                print(f"Original image not found for {image_path}")
                return False

            original_image = cv2.imread(original_image_path)

            # Create a mask with the circle
            mask = np.zeros_like(original_image)
            cv2.circle(mask, (center_x, center_y), radius, (255, 255, 255), thickness=-1)

            # Apply the mask to the original image
            result_image = cv2.bitwise_and(original_image, mask)

            # Convert grayscale image to BGR for colored drawing
            image_color = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

            # Draw the outer circle in red
            cv2.circle(image_color, (center_x, center_y), radius, (0, 0, 255), 4)  # Red color, thicker line
            # Draw the center of the circle
            cv2.circle(image_color, (center_x, center_y), 2, (0, 255, 0), 3)

            # Save the result image with the circle
            output_image_path = os.path.join(output_dir, os.path.basename(image_path))
            if not os.path.exists(output_image_path):
                cv2.imwrite(output_image_path, image_color)

            # Save the result image with only the circle
            result_output_path = os.path.join(output_dir, f"circle_{os.path.basename(image_path)}")
            if not os.path.exists(result_output_path):
                cv2.imwrite(result_output_path, result_image)

            return True
        else:
            print(f"No valid circles were found in {os.path.basename(image_path)}")
    else:
        print(f"No circles were found in {os.path.basename(image_path)}")

    return False

def is_circle_valid(image, center_x, center_y, radius):
    # Add any additional criteria for validating circles here
    # For example, check if the circle is within the image bounds
    h, w = image.shape
    return True

# Path to the images directory
images_directory = '/content/mydrive/Othercomputers/My PC/Thesis/roboflowV6_1169/segmented_with_lbp_mandarins'
output_directory = '/content/mydrive/Othercomputers/My PC/Thesis/roboflowV6_1169/circeled_images_25.7'
os.makedirs(output_directory, exist_ok=True)

# Process all images starting with "LBP" in the directory
image_files = [f for f in os.listdir(images_directory) if f.startswith("LBP") and f.endswith(".JPG")]
total_images = len(image_files)
total_circles = 0

for image_file in image_files:
    image_path = os.path.join(images_directory, image_file)
    detected = detect_circular_shape(image_path, output_directory, scale_factor=0.2, min_radius=1200, max_radius=1700)
    if detected:
        total_circles += 1

# Final summary
print(f"\nTotal number of images: {total_images}")
print(f"Total circles detected: {total_circles}")

# Count the number of images in circeled_images that don't start with "LBP"
circled_image_files = [f for f in os.listdir(output_directory) if not f.startswith("LBP")]
non_lbp_images_count = len(circled_image_files)

print(f"Total number of non-LBP images in circeled_images: {non_lbp_images_count}")


Searching for /content/mydrive/Othercomputers/My PC/Thesis/roboflowV6_1169/original/bounded/1D3A4548_1.JPG
original_image_path /content/mydrive/Othercomputers/My PC/Thesis/roboflowV6_1169/original/bounded/1D3A4548_1.JPG
No circles were found in LBP_Thresh_1D3A4805_1.JPG
No circles were found in LBP_Thresh_1D3A4805_2.JPG
No circles were found in LBP_Thresh_1D3A4805_3.JPG
No circles were found in LBP_Thresh_1D3A4814_1.JPG
Searching for /content/mydrive/Othercomputers/My PC/Thesis/roboflowV6_1169/original/bounded/1D3A4815_1.JPG
original_image_path /content/mydrive/Othercomputers/My PC/Thesis/roboflowV6_1169/original/bounded/1D3A4815_1.JPG
Searching for /content/mydrive/Othercomputers/My PC/Thesis/roboflowV6_1169/original/bounded/1D3A4816_1.JPG
original_image_path /content/mydrive/Othercomputers/My PC/Thesis/roboflowV6_1169/original/bounded/1D3A4816_1.JPG
No circles were found in LBP_Thresh_1D3A4825_1.JPG
Searching for /content/mydrive/Othercomputers/My PC/Thesis/roboflowV6_1169/original/b

KeyboardInterrupt: 

22.7 - try to improve the validation of the circle found, not based on the largest circle by on the texture on more

In [ ]:
import cv2
import numpy as np
import os

def find_original_image(image_name):
    original_image_name = image_name.replace("LBP_Thresh_", "")
    search_paths = [
        '/content/mydrive/Othercomputers/My PC/Thesis/roboflowV6_1169/original/bounded'
    ]

    for path in search_paths:
        original_image_path = os.path.join(path, original_image_name)
        print(f"Searching for {original_image_path}")
        if os.path.exists(original_image_path):
            return original_image_path
    return None

def is_circle_within_bounds(image, center_x, center_y, radius):
    h, w = image.shape
    return center_x - radius > 0 and center_x + radius < w and center_y - radius > 0 and center_y + radius < h

def detect_circular_shape(image_path, output_dir, scale_factor=0.2, min_radius=300, max_radius=400):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Error loading image: {image_path}")
        return False

    resized_image = cv2.resize(image, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)
    enhanced_image = cv2.equalizeHist(resized_image)
    blurred_image = cv2.GaussianBlur(enhanced_image, (9, 9), 2)

    circles = cv2.HoughCircles(
        blurred_image,
        cv2.HOUGH_GRADIENT,
        dp=1,
        minDist=100,
        param1=100,
        param2=50,
        minRadius=int(min_radius * scale_factor),
        maxRadius=int(max_radius * scale_factor)
    )

    if circles is not None:
        circles = np.uint16(np.around(circles))
        valid_circles = []

        for circle in circles[0, :]:
            center_x = int(circle[0] / scale_factor)
            center_y = int(circle[1] / scale_factor)
            radius = int(circle[2] / scale_factor)

            if is_circle_within_bounds(image, center_x, center_y, radius):
                valid_circles.append((center_x, center_y, radius))

        if valid_circles:
            # Select the first valid circle (instead of the largest)
            center_x, center_y, radius = valid_circles[0]

            original_image_name = os.path.basename(image_path).replace("LBP_Thresh_", "")
            original_image_path = find_original_image(original_image_name)
            print("original_image_path", original_image_path)
            if original_image_path is None:
                print(f"Original image not found for {image_path}")
                return False

            original_image = cv2.imread(original_image_path)

            mask = np.zeros_like(original_image)
            cv2.circle(mask, (center_x, center_y), radius, (255, 255, 255), thickness=-1)
            result_image = cv2.bitwise_and(original_image, mask)
            image_color = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

            cv2.circle(image_color, (center_x, center_y), radius, (0, 0, 255), 4)
            cv2.circle(image_color, (center_x, center_y), 2, (0, 255, 0), 3)

            output_image_path = os.path.join(output_dir, os.path.basename(image_path))
            if not os.path.exists(output_image_path):
                cv2.imwrite(output_image_path, image_color)

            result_output_path = os.path.join(output_dir, f"circle_{os.path.basename(image_path)}")
            if not os.path.exists(result_output_path):
                cv2.imwrite(result_output_path, result_image)

            return True
        else:
            print(f"No valid circles were found in {os.path.basename(image_path)}")
    else:
        print(f"No circles were found in {os.path.basename(image_path)}")

    return False

# Path to the images directory
images_directory = '/content/mydrive/Othercomputers/My PC/Thesis/roboflowV6_1169/segmented_with_lbp_mandarins'
output_directory = '/content/mydrive/Othercomputers/My PC/Thesis/roboflowV6_1169/improved_circeled_images'
os.makedirs(output_directory, exist_ok=True)

# Process all images starting with "LBP" in the directory
image_files = [f for f in os.listdir(images_directory) if f.startswith("LBP") and f.endswith(".JPG")]
total_images = len(image_files)
total_circles = 0

for image_file in image_files:
    image_path = os.path.join(images_directory, image_file)
    detected = detect_circular_shape(image_path, output_directory, scale_factor=0.2, min_radius=1200, max_radius=1700)
    if detected:
        total_circles += 1

# Final summary
print(f"\nTotal number of images: {total_images}")
print(f"Total circles detected: {total_circles}")

# Count the number of images in circeled_images that don't start with "LBP"
circled_image_files = [f for f in os.listdir(output_directory) if not f.startswith("LBP")]
non_lbp_images_count = len(circled_image_files)

print(f"Total number of non-LBP images in circeled_images: {non_lbp_images_count}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/mydrive/Othercomputers/My PC/Thesis/roboflowV6_1169/segmented_with_lbp_mandarins'